In [152]:
# 1. Loss coefficients by land use type and intensity (from True Price table)
loss_coefficient = {              
    "natural_forest_minimal": 0.15,
    "natural_forest_light": 0.30,
    "natural_forest_intense": 0.50,
    "plantation_forest_minimal": 0.70,
    "plantation_forest_light": 0.75,
    "plantation_forest_intense": 0.80,
    "cropland_minimal": 0.60,
    "cropland_light": 0.70,
    "cropland_intense": 0.90,
    "pasture_light": 0.20,
    "pasture_moderate": 0.40,
    "pasture_heavy": 0.70,

# Loss coefficients estimates
    "food_forest_young": 0.70,
    "food_forest_developing": 0.50,
    "food_forest_mature": 0.30,
}

# 2. Biome ESS values (€ per MSA·ha·year)
biome_ess_values = {
    'grassland': 2427,
    'temperate_forest': 1014,
    'wetland_inland': 14871,
    'wetland_coastal': 10939,
    'woodlands': 1369
}

# 3. Biome shares for the Netherlands
biome_share = {
    'grassland': 0.68,
    'temperate_forest': 0.14,
    'wetland_inland': 0.08,
    'wetland_coastal': 0.4,
    'woodlands': 0.6
}

# 4. Restoration costs (€ per MSA·ha), from True Price Table 4, adjusted for inflation to 2024
restoration_costs = {
    'grassland': 336.97,
    'temperate_forest': 4470.15,
    'wetland_inland': 19807.92,
    'wetland_coastal': 7902.03,
    'woodlands': 1702.26
}


In [ ]:
def get_food_forest_class(years_since_conversion):
    if years_since_conversion <= 5:
        return "food_forest_young"
    elif years_since_conversion <= 14:
        return "food_forest_developing"
    else:
        return "food_forest_mature"


def compute_land_use_metrics(yield_kg_per_ha, biome_type, land_use_type, intensity, years_since_conversion):
    # Calculate land use per kg product
    use = 1 / yield_kg_per_ha  # ha·yr per kg

    # Select the right loss coefficient key
    if land_use_type == "food_forest":
        intensity = get_food_forest_class(years_since_conversion).replace("food_forest_", "")
        key = f"food_forest_{intensity}"
    else:
        key = f"{land_use_type}_{intensity}"

    loss = loss_coefficient.get(key, None)
    msa = 1 - loss
    biome_fraction = biome_share.get(biome_type, None)
    ess_value = biome_ess_values.get(biome_type, 0)
    restoration_value = restoration_costs.get(biome_type, 0)

    if msa is None:
        raise ValueError(f"Invalid land use type or intensity: {key}")
    if biome_fraction is None:
        raise ValueError(f"Invalid biome type: {biome_type}")

    # Land use per kg
    land_ob = use * biome_fraction * loss
    land_cb = land_ob / years_since_conversion if years_since_conversion > 0 else 1
    land_use_cost = land_ob * ess_value
    land_use_change_cost = land_cb * restoration_value

    # Land use per ha (multiply by yield)
    land_use_cost_per_ha = land_use_cost * yield_kg_per_ha
    land_use_change_cost_per_ha = land_use_change_cost * yield_kg_per_ha if land_use_change_cost is not None else None

    return {
        "MSA value (biodiversity retained)": msa,
        "Biome Share": biome_fraction,
        "LAND-Ob (MSA.ha.yr/kg)": round(land_ob, 8),
        "LAND-Cb (MSA.ha/kg)": round(land_cb, 8) if land_cb is not None else "N/A",
        "Land Use Cost (€ per kg)": f"{land_use_cost:.4f}",
        "Land Use Change Cost (€ per kg)": f"{land_use_change_cost:.4f}" if land_use_change_cost is not None else "N/A",
        "Land Use Cost (€ per ha)": f"{land_use_cost_per_ha:.2f}",
        "Land Use Change Cost (€ per ha)": f"{land_use_change_cost_per_ha:.2f}" if land_use_change_cost_per_ha is not None else "N/A"
    }


# Inputs
yield_per_hectare = 500       # Example food forest, kg/ha/yr
biome = "grassland"
years = 20

food_forest = compute_land_use_metrics(
    yield_kg_per_ha=yield_per_hectare,
    biome_type=biome,
    land_use_type="food_forest",
    intensity=None,
    years_since_conversion=years,
)

conventional_agriculture = compute_land_use_metrics(
    yield_kg_per_ha=64244,
    biome_type=biome,
    land_use_type="cropland",
    intensity="intense",
    years_since_conversion=years,
)

food_forest, conventional_agriculture


TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'